# Playground

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append("./src")

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
from torch.utils.data import DataLoader

from dataset import *
from plots import *
from models import *

## Metadata Summary

In [ ]:
metadata = load_metadata()

In [ ]:
metadata.head()

In [ ]:
metadata.shape

In [ ]:
plot_MOA_distribution(metadata)

In [ ]:
plot_treatment_heatmap(metadata)

## Tiny - Subset of Dataset

In [ ]:
tiny_metadata = metadata[metadata["Multi_Cell_Image_Name"] == "Week10_200907_F02_s1_w14631241C-4FA2-4BC9-8693-D7D268CAEE82"]
#tiny_metadata = metadata[metadata["Multi_Cell_Image_Name"] == "B02_s1_w16F89C55C-7808-4136-82E4-E066F8E3CB10"]

In [ ]:
images = load_images_from_metadata(tiny_metadata)
images = normalize_channel_wise(images)

print(f"{images.shape[0]} images")

In [ ]:
plot_treatment_heatmap(tiny_metadata)

In [ ]:
img1 = images[0]

In [ ]:
plot_image(img1)

In [ ]:
plot_channels(img1)

## Stratified - Subset of Dataset

In [ ]:
blacklist = [("Eg5 inhibitors", 0.1), ("Microtubule destabilizers", 0.3), ("Cholesterol-lowering", 6.0)]
stratified = stratify_metadata(metadata, 60, blacklist=blacklist)

In [ ]:
images = load_images_from_metadata(stratified)
images = normalize_channel_wise(images)

print(f"{images.shape[0]} images")

In [ ]:
plot_treatment_heatmap(stratified)

## Model - Training

In [ ]:
images = normalized_to_zscore(images)

In [ ]:
print(f"{images.shape[0]} images")

In [ ]:
cropped_images = view_cropped_images(images)
train(cropped_images, epochs=20, epoch_sample_times=2)

## Model - Load pretrained

In [ ]:
unet = UNet()
unet.load_state_dict(torch.load("./models/DDPM_Unconditional/ckpt.pt"))
diffusion = Diffusion()
sampled_images = diffusion.sample(unet, n=1)

## Results

In [ ]:
epoch_images, epochs = load_epoch_images()
plot_epoch_sample_series(epoch_images, epochs)

In [ ]:
plot_image(epoch_images[-1][0])